In [ ]:
import pandas as pd
import networkx as nx


In [ ]:
dataNode = pd.read_csv('../_data/Node.csv')                       
dataLink = pd.read_csv('../_data/Link.csv')                       


In [ ]:
associationStrength= {"r_cert": 4, 
                      "r_subdomain": 4,
                      "r_request_jump":4 ,
                      "r_dns_a": 4,
                      "r_whois_email":3,
                      "r_whois_phone":3,
                      "r_whois_name":3,
                      "r_cert_chain":2,
                      "r_cname":2,
                      "r_asn":1,
                      "r_cidr":1,}

dataLink['associationStrength'] = dataLink['relation'].map(associationStrength)



In [ ]:
# dataLink.head(5)

In [ ]:
# dataNode.head(5)

In [ ]:
# sd_rows = dataNode[dataNode['id'].str.contains('Domain_c58c149eec59bb14b0c102a0f303d4c20366926b5c3206555d2937474124beb9')]
# sd_rows

In [ ]:
# sa_relation = dataLink[dataLink['source'].str.contains('Domain_f3554b666038baffa5814c319d3053ee2c2eb30d31d0ef509a1a463386b69845')]
# sa_relation

In [ ]:

G = nx.Graph()
for i in range(len(dataLink)):
    source = dataLink.loc[i, "source"]
    target = dataLink.loc[i, "target"]
    strength = dataLink.loc[i, "associationStrength"]
    G.add_edge(source, target, weight=strength)

In [ ]:
# 定义关联强度阈值
STRONG_THRESHOLD = 4
GENERAL_THRESHOLD = 2

# 定义起始节点
start_nodes = ["Domain_f3554b666038baffa5814c319d3053ee2c2eb30d31d0ef509a1a463386b69845",
                "Domain_c58c149eec59bb14b0c102a0f303d4c20366926b5c3206555d2937474124beb9"]

# start_nodes = ["Domain_f3554b666038baffa5814c319d3053ee2c2eb30d31d0ef509a1a463386b69845"]



In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:

# 函数：根据起始节点和业务规则找到子图
def find_subgraph(start_nodes, G, strong_threshold, general_threshold):
    # 初始化子图
    subgraph = nx.Graph()

    # 遍历起始节点
    for start_node in start_nodes:
        # 3 跳以内节点
        for neighbor in nx.neighbors(G, start_node):
            subgraph.add_node(neighbor)
            subgraph.add_edge(start_node, neighbor)
            for neighbor2 in nx.neighbors(G, neighbor):
                # 关联强度较强的边
                if G[neighbor][neighbor2]['weight'] >= strong_threshold:
                    subgraph.add_node(neighbor2)
                    subgraph.add_edge(neighbor, neighbor2)
                # 关联强度一般的边
                elif G[neighbor][neighbor2]['weight'] >= general_threshold:
                    subgraph.add_node(neighbor2)
                    subgraph.add_edge(neighbor, neighbor2)
                    # 2 跳以内节点
                    for neighbor3 in nx.neighbors(G, neighbor2):
                        subgraph.add_node(neighbor3)
                        subgraph.add_edge(neighbor2, neighbor3)

    # 返回子图
    return subgraph

# 找到子图
subgraph = find_subgraph(start_nodes, G, STRONG_THRESHOLD, GENERAL_THRESHOLD)

# 输出子图信息
print(f"子图节点数：{subgraph.number_of_nodes()}")
print(f"子图边数：{subgraph.number_of_edges()}")

In [ ]:
# import json

# # 将节点转换为 JSON 格式
# nodes = []
# for node in subgraph.nodes():
#     nodes.append({"id": node})

# # 将边转换为 JSON 格式
# links = []
# for source, target, data in subgraph.edges(data=True):
#     links.append({"source": source, "target": target})

# # 将子图转换为 JSON 格式
# data = {"nodes": nodes, "links": links}

# # 存储 JSON 文件
# with open("subgraph.json", "w") as f:
#     json.dump(data, f)